In [1]:
import numpy as np, pandas as pd

In [2]:
class GDAMulticlass(object):
    
    def __init__(self, features, target):
        self.features=features
        self.target=target
        
    def train(self):
        combined=pd.concat([self.features, self.target], axis=1)
        self.mu_s=[combined[combined['target']==j].drop('target', axis=1).mean(axis=0)\
                   for j in range(len(self.target.unique()))]
        self.phi_s=(1.0/len(self.target))*np.array([self.target[self.target==j].count() \
                                    for j in range(len(self.target.unique()))])
        sigma=np.matrix(np.zeros([self.features.shape[1], self.features.shape[1]]))
        for i in range(self.target.shape[0]):
            sigma += np.dot(np.matrix(self.features.iloc[i, :]-\
            self.mu_s[self.target.iloc[i]]).T, \
            np.matrix(self.features.iloc[i, :]-self.mu_s[self.target.iloc[i]]))
        
        self.sigma=(1.0/self.target.shape[0])*sigma
        
    def P_y(self, y, phi_s):
        return phi_s[y]
    
    def P_x_given_y(self, sigma, x, mu):
        comp1 = 1.0/(np.sqrt((2*np.pi)**self.features.shape[1]) * np.sqrt(\
                                                    np.linalg.det(sigma)))
        comp2 = float(np.exp(np.dot(-0.5*np.dot(x-mu, np.linalg.inv(sigma)), x-mu)))
        return comp1*comp2
        
    def predict(self, X):
        predictions=[]
        for i in range(X.shape[0]):
            Prob=[self.P_x_given_y(self.sigma, X.iloc[i, :], self.mu_s[j])*self.P_y(j,\
                    self.phi_s) for j in range(len(self.target.unique()))]
            
            predictions.append(np.argmax(Prob))
            
        return np.array(predictions)
            